In [1]:
import numpy as np
import tensorflow as tf
import tflearn
from six.moves import cPickle as pickle
from sklearn import linear_model

In [2]:
pickle_file = 'nusm.pickle'
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (62332, 42), (62332,))
('Validation set', (10000, 42), (10000,))
('Test set', (10000, 42), (10000,))


In [3]:
num_labels = 2

def reformat(labels):
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return labels

train_labels = reformat(train_labels)
test_labels = reformat(test_labels)
valid_labels = reformat(valid_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (62332, 42), (62332, 2))
('Validation set', (10000, 42), (10000, 2))
('Test set', (10000, 42), (10000, 2))


In [4]:
input_layer = tflearn.input_data(shape=[None, 42])
dense1 = tflearn.fully_connected(input_layer, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout1 = tflearn.dropout(dense1, 0.8)
dense2 = tflearn.fully_connected(dropout1, 64, activation='tanh',
                                 regularizer='L2', weight_decay=0.001)
dropout2 = tflearn.dropout(dense2, 0.8)
softmax = tflearn.fully_connected(dropout2, 2, activation='softmax')

# Regression using SGD with learning rate decay and Top-3 accuracy
sgd = tflearn.Adam(learning_rate=0.01, beta1=0.99)
# top_k = tflearn.metrics.Top_k(k=1)
acc = tflearn.metrics.Accuracy()
net = tflearn.regression(softmax, optimizer=sgd, metric=acc,
                         loss='categorical_crossentropy')

# Training
model = tflearn.DNN(net, tensorboard_verbose=0)
model.fit(train_dataset, train_labels, n_epoch=20, validation_set=(valid_dataset, valid_labels),
          show_metric=True, run_id="dense_model")

Training Step: 19479  | total loss: 0.57733 | time: 4.980s
| Adam | epoch: 020 | loss: 0.57733 - acc: 0.7150 -- iter: 62272/62332
Training Step: 19480  | total loss: 0.57465 | time: 6.062s
| Adam | epoch: 020 | loss: 0.57465 - acc: 0.7154 | val_loss: 0.55725 - val_acc: 0.7043 -- iter: 62332/62332
--


In [5]:
model.evaluate(X=test_dataset, Y=test_labels)

[0.70009999999999994]

In [6]:
test_labels


array([[ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]], dtype=float32)